# Analiza stabilnosti i pouzdanosti modela linearne regresije

Ovaj notebook je namenjen kratkoj prezentaciji (≈10 min). Većina implementacije je u `Core/` Python fajlovima, a ovde samo pozivamo funkcije, prikazujemo ključne ispise i grafikone.


## 1. Učitavanje i osnovna podešavanja

Uvozimo funkcije iz projekta i definišemo putanju ka `train.csv`, target i izabrane feature-e.


In [ ]:
import os, sys
import pandas as pd

# Dodaj root projekta u sys.path (da možemo da importujemo Core modul)
ROOT = os.path.abspath('..')
if ROOT not in sys.path:
    sys.path.append(ROOT)

from Core.priprema import pripremi_podatke
from Core.run_models import train_all_models, stabilnost_svih_metoda, eksperiment_sa_sumom
from Core.notebook_helpers import prikazi_rezultate, koeficijenti_u_tabelu, prikazi_stabilnost, prikazi_sum

PUTANJA_TRAIN = os.path.join('..', 'Podaci', 'train.csv')
CILJ = 'SalePrice'
FEATURES = ['GrLivArea','OverallQual','YearBuilt','TotalBsmtSF','GarageArea']

# podešavanja
NACIN_NAN = 'dropna'          # ili 'interpolate'
STANDARDIZACIJA = True

**Kratko:** Koristimo 5 numeričkih feature-a koji imaju jasnu vezu sa cenom kuće. Standardizaciju uključujemo zbog regularizacije.


## 2. Priprema podataka (split + standardizacija)

Radimo podelu 80/20 (train/val) i standardizaciju feature-a (fit samo na trening skupu).

In [ ]:
out = pripremi_podatke(
    putanja_train_csv=PUTANJA_TRAIN,
    cilj=CILJ,
    feature_kolone=FEATURES,
    nacin_nedostajuce=NACIN_NAN,
    dummy_flag=False,
    train_size=0.8,
    random_state=42,
    standardizacija_flag=STANDARDIZACIJA
)

if STANDARDIZACIJA:
    df, x, y, x_train, x_val, y_train, y_val, srednje, std = out
else:
    df, x, y, x_train, x_val, y_train, y_val = out

display(df[[CILJ] + FEATURES].head())
print('Train shape:', x_train.shape, 'Val shape:', x_val.shape)

**Zaključak:** Imamo trening i validacioni skup, a feature-i su na istoj skali (što je bitno za L1/L2 regularizaciju).


## 3. Treniranje modela

Treniramo 5 modela (ručne implementacije): OLS, Ridge, Lasso, Elastic Net i Huber. Dobijamo tabelu metrika i koeficijente.


In [ ]:
results_df, models, data_out = train_all_models(
    putanja_train_csv=PUTANJA_TRAIN,
    cilj=CILJ,
    feature_kolone=FEATURES,
    nacin_nedostajuce=NACIN_NAN,
    dummy_flag=False,
    standardizacija_flag=STANDARDIZACIJA,
    random_state=42,
    # hiperparametri (primer)
    ridge_alfa=0.1,
    lasso_alfa=0.1,
    en_alfa=0.1,
    en_l1_ratio=0.5,
    huber_delta=1.0,
    # iteracije
    gd_epochs=300,
    cd_epochs=30,
)

prikazi_rezultate(results_df)

# Koeficijenti (sažeto)
coef_df = koeficijenti_u_tabelu(models, FEATURES)
display(coef_df)

**Napomena:** RMSE je izražen u dolarima. Pošto `SalePrice` ima širok raspon, RMSE od nekoliko desetina hiljada može biti očekivan. Regularizacija često stabilizuje koeficijente i smanjuje varijansu.


## 4. Stabilnost koeficijenata (više random split-ova)

Ponavljamo proces obuke na više slučajnih podela i posmatramo standardnu devijaciju koeficijenata. Manja std = stabilniji model.


In [ ]:
stab_df = stabilnost_svih_metoda(
    putanja_train_csv=PUTANJA_TRAIN,
    cilj=CILJ,
    feature_kolone=FEATURES,
    repeats=10,
    nacin_nedostajuce=NACIN_NAN,
    standardizacija_flag=STANDARDIZACIJA,
    gd_epochs=300,
    cd_epochs=30
)

prikazi_stabilnost(stab_df)

**Zaključak:** Regularizovani modeli (posebno Ridge i Elastic Net) tipično pokazuju manje oscilacije koeficijenata u odnosu na OLS.


## 5. Eksperiment sa kontrolisanim šumom

U trening target dodajemo Gauss-ov šum:  σ = std(y_train) × noise_sigma. Posmatramo kako rast šuma utiče na RMSE.


In [ ]:
noise_df = eksperiment_sa_sumom(
    putanja_train_csv=PUTANJA_TRAIN,
    cilj=CILJ,
    feature_kolone=FEATURES,
    noise_sigma_values=(0.0, 0.05, 0.1, 0.2),
    random_state=42,
    nacin_nedostajuce=NACIN_NAN,
    standardizacija_flag=STANDARDIZACIJA,
    gd_epochs=300,
    cd_epochs=30
)

prikazi_sum(noise_df)

## 6. Kratak zaključak

- OLS može imati dobre rezultate, ali koeficijenti često više osciluju.
- Ridge/Elastic Net obično daju stabilnije koeficijente (manja varijansa).
- Lasso može „gasi“ neke koeficijente (sparsity), što pomaže interpretabilnosti.
- Huber je robusniji kada postoje outlier-i.
- Sa rastom šuma, RMSE raste; regularizovani modeli često degradiraju „mirnije“.
